In [ ]:
%env KERAS_BACKEND=cntk
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import apdm
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, LSTM, TimeDistributed 
from keras.optimizers import Adam,rmsprop
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pyfirmata
import keyboard
import math
import socket
from datetime import datetime
import time




# model = load_model('model_6axis_Opal_samplingrate20_lockback50_0418.h5')
# model = load_model('model_test_0828.h5')
model = load_model('model_test_total_0225.h5')
model.summary()
# model.layers[0].unroll=True



# 連接至Predict_HS system
predict_server_address = ('192.168.1.2', 12345)  

# 创建一个TCP/IP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 连接到Predict_HS服务器
client_socket.connect(predict_server_address)

# 发送信号到Unity服务器
def send_signal(signal):
    message = signal.encode()  # 将信号转换为字节串
    client_socket.sendall(message)  # 发送信号
    # 初始化 Socket 非阻塞模式
    client_socket.setblocking(False)

    # 設置 Socket 發送緩衝區大小
    client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 4096)
    


#set parameter
data_5610=np.array([[1,2,3,4,5,6,7]])
data_5611=np.array([[1,2,3,4,5,6,7]])
data_5617=np.array([[1,2,3,4,5]])
R_HeelStrike_Detect_Time=np.array([[1]])
L_HeelStrike_Detect_Time=np.array([[1]])
tl=0
tt=0
t_before_write=0
t_after_write=0
getdata=1
samplingtime=0.045
detect_state=1
hs_0=0
hs_1=0
i2_1=0
i2_0=0
pelvic_angle=np.array([[100]])
rotation_amplitude=np.array([[200]])
paretic_side=0 #Paretic side setting
hs_index_last=0
hs_index_now=0
pelvic_rotation_thres=12 # init third rule threshold
TR_trigger_num=0

#test
data_5610_2 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_5617_2 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_7750 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_7770 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_11141 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])

data_www = np.zeros((50,6))
block_count = 0
block_time = 0.3

aaaa=0
bbbb = np.array([[1,2,3,4]])
cccc = np.array([[1]])

print("Initialized")

In [ ]:
num_aps = apdm.uintArray(1)
return_code = apdm.apdm_ap_get_num_access_points_on_host1(num_aps)
print "return code is " + str(return_code)
print "Number of APs on host is " + str(num_aps)

ap_handle = apdm.apdm_ap_allocate_handle()
context = apdm.apdm_ctx_allocate_new_context()
#print context

if context == None:
    raise Exception("unable to allocate new context")
    
r = apdm.apdm_ctx_open_all_access_points(context)
if r != apdm.APDM_OK:   
    raise Exception("unable to open all access points")
    
r = apdm.apdm_ctx_sync_record_list_head(context)
if r != apdm.APDM_OK:
    raise Exception("unable to sync record head list")

raw_record = apdm.apdm_record_t()

# r = apdm.apdm_ctx_flush_ap_fifos(context)
# if r != apdm.APDM_OK:   
#     raise Exception("Failed to flush AP fifos\n")

numSensors = apdm.uintArray(1)
retCode = apdm.apdm_ctx_get_expected_number_of_sensors2(context, numSensors)
device_ids = []
for x in range(numSensors[0]):
    device_ids.append(apdm.apdm_ctx_get_device_id_by_index(context, x))

numdevices=len(device_ids)

ti=time.clock()  
print("start processing")
while True:
#     aaaa=aaaa+1;
#     if aaaa==25:
#         print(pelvic_now_angle)
#         print(bbbb)
#         aaaa=0
    if keyboard.is_pressed('v'):
        print('Process stop!')
        break
    getdata=0
    while getdata==0 or cpt!=numdevices:
        cpt=0
        r = apdm.apdm_ctx_get_next_access_point_record_list(context)
        for device_id in device_ids:
            r = apdm.apdm_ctx_extract_data_by_device_id(context, device_id,raw_record);
            if r == apdm.APDM_NO_MORE_DATA:
                #print("Got no more data, continue")
                apdm.apdm_usleep(2000)
                getdata=0
                break
            elif r != apdm.APDM_OK:
                print("ERROR: apdm_get_next_access_point_record_list return an unexpected error");
                break
            else:
                data_now=np.array([[tl,raw_record.accl_x_axis_si,raw_record.accl_y_axis_si,raw_record.accl_z_axis_si,raw_record.gyro_x_axis_si,raw_record.gyro_y_axis_si,raw_record.gyro_z_axis_si]])
                data_now_orientation=np.array([[raw_record.orientation_quaternion0,raw_record.orientation_quaternion1,raw_record.orientation_quaternion2,raw_record.orientation_quaternion3]])
                pelvic_now_angle=np.array([[math.asin(-2*(raw_record.orientation_quaternion0*raw_record.orientation_quaternion2+raw_record.orientation_quaternion1*raw_record.orientation_quaternion3))*180/math.pi]])
                if device_id==5610:
                    data_5610=np.vstack([data_5610,data_now])
                    
                    #將新收到data_5610的data從1X7 array轉成一維，並用逗號分隔每一元素，並發送到predict那
                    latest_data_5610  = data_now.flatten()
                    message_data_5610 = "data_5610:" + ",".join(map(str, latest_data_5610)) + "\n"
                    client_socket.sendall(message_data_5610.encode())  # 傳送資料
                    print("{} {}".format(datetime.now(), "send message_data_5610"))

                    
                    data_5610_2 = np.vstack([data_5610_2, np.hstack([data_now, data_now_orientation]) ])
                
                elif device_id==5611:
                    data_5611=np.vstack([data_5611,data_now])
                    
                    #將新收到data_5611的data從1X7 array轉成一維，並用逗號分隔每一元素，並發送到predict那
                    latest_data_5611  = data_now.flatten()
                    #message_data_5611 = "data_5611:" + ",".join(map(str, latest_data_5611)) + "\n"
                    #client_socket.sendall(message_data_5611.encode())  # 傳送資料

                elif device_id==5617:
#                     bbbb = data_now_orientation
#                     cccc = pelvic_now_angle
                    pelvic_angle=np.vstack([pelvic_angle,pelvic_now_angle])
                    data_5617_2 = np.vstack([data_5617_2, np.hstack([data_now, data_now_orientation]) ])
                elif device_id==7750:
                    data_7750 = np.vstack([data_7750, np.hstack([data_now, data_now_orientation]) ])
                elif device_id==7770:
                    data_7770 = np.vstack([data_7770, np.hstack([data_now, data_now_orientation]) ])
                elif device_id==11141:
                    data_11141 = np.vstack([data_11141, np.hstack([data_now, data_now_orientation]) ])
                getdata=1
                cpt=cpt+1
        
    if len(pelvic_angle)==2666: #after A-before-phase 120 seconds, decide 3rd rule threshold for B-phase
        pelvic_rotation_thres = np.mean(rotation_amplitude[2:len(rotation_amplitude)-1,0]) - 0.5*np.std(rotation_amplitude[2:len(rotation_amplitude)-1,0], ddof=1)
    
    
    if getdata==1 and len(data_5610)>50:
#             print("detecting")
#             and max(data_5610[len(data_5610)-20:len(data_5610),5])*180/3.14>5
#         if detect_state==1 and block_count*samplingtime>block_time and np.std(data_5610[len(data_5610)-50:len(data_5610),5],ddof=1)>0.3:
        if detect_state==1 and block_count*samplingtime>block_time:
#             print("detecting1")
            input=preprocessing.scale(data_5610[len(data_5610)-50:len(data_5610),1:7])
            input=input.reshape(1,50,6)
            hs_1_level=model.predict(input)
#             if sum(hs_1_level[0,49])>0.015 and data_5610[len(data_5610)-1,5]*180/3.14<-1:  # HS的level高於threshold且角速度低於threshold
#             if hs_1_level[0,49,0]>0.015 and data_5610[len(data_5610)-1,5]*180/3.14<-10 and max(data_5610[len(data_5610)-20:len(data_5610),5])*180/3.14>10:  # HS的level高於threshold且角速度低於threshold
            if hs_1_level[0,49,0]>0.015 and data_5610[len(data_5610)-1,5]*180/3.14<-10:
                i1_1=i2_1
                i2_1=data_5610[len(data_5610)-1,0]
                if i2_1-i1_1>0.1:
#                     print("Rheelstrik!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    #t_before_write =time.clock()
                    #print(t_before_write)
                    R_HeelStrike_Detect_Time = np.vstack([R_HeelStrike_Detect_Time, i2_1])
                    
                    #當右腳HS時，發送訊號要開始預測左腳哪時HS
                    message = "ready to predict left leg" + "\n"
                    print("{} {}".format(datetime.now(), message))
                    client_socket.sendall(message.encode())
                    
#                     hs_1=hs_1+1  
                    if paretic_side==detect_state:
                        hs_index_last=hs_index_now
                        hs_index_now=len(data_5610) 
                        pelvic_rotation=max(pelvic_angle[hs_index_last:hs_index_now,0])-min(pelvic_angle[hs_index_last:hs_index_now,0])
                        rotation_amplitude=np.vstack([rotation_amplitude,pelvic_rotation])
                        #t_after_write =time.clock()
                        #print(t_after_write)
                        if pelvic_rotation<pelvic_rotation_thres:
#                             board.digital[TR1pin].write(1)
#                             print("TR",pelvic_rotation)
                            print(pelvic_rotation)
                            TR_trigger_num=TR_trigger_num+1
                        else:
#                             print("777")
                            print(pelvic_rotation)
#                             board.digital[TR1pin].write(0)
                    detect_state=0   #換腳偵測
                    block_count = 0
    
#         elif detect_state==0 and block_count*samplingtime>block_time and np.std(data_5611[len(data_5611)-50:len(data_5611),5],ddof=1)>0.3:
        elif detect_state==0 and block_count*samplingtime>block_time:
            input=preprocessing.scale(data_5611[len(data_5611)-50:len(data_5611),1:7])
            input=input.reshape(1,50,6)
            input[0,:,1]=-input[0,:,1]
            input[0,:,3]=-input[0,:,3]
            input[0,:,5]=-input[0,:,5]
            hs_0_level=model.predict(input)
#             if sum(hs_0_level[0,49])>0.012 and data_5611[len(data_5611)-1,5]*180/3.14<-1:  # HS的level高於threshold且角速度低於threshold
#             if hs_0_level[0,49,0]>0.015 and data_5611[len(data_5611)-1,5]*180/3.14<-10 and max(data_5611[len(data_5611)-20:len(data_5611),5])*180/3.14>3:  # HS的level高於threshold且角速度低於threshold
            if hs_0_level[0,49,0]>0.015 and data_5611[len(data_5611)-1,5]*180/3.14<-10:  # HS的level高於threshold且角速度低於threshold
                i1_0=i2_0
                i2_0=data_5611[len(data_5611)-1,0]
                if i2_0-i1_0>0.1:
#                     print("Lheelstrike!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    L_HeelStrike_Detect_Time = np.vstack([L_HeelStrike_Detect_Time, i2_0])
                   
                    #當左腳HS時，發送訊號要開始預測右腳哪時HS
                    message = "ready to predict right leg" + "\n"
                    execution_time = time.time()
                    client_socket.sendall(message.encode())
                    print("{} {} {:.6f}".format(datetime.now(), message, execution_time))
                
#                     hs_0=hs_0+1    
                    if paretic_side==detect_state:
                        hs_index_last=hs_index_now
                        hs_index_now=len(data_5611) 
                        pelvic_rotation=max(pelvic_angle[hs_index_last:hs_index_now,0])-min(pelvic_angle[hs_index_last:hs_index_now,0])
                        rotation_amplitude=np.vstack([rotation_amplitude,pelvic_rotation])
                        if pelvic_rotation<pelvic_rotation_thres:
#                             print("TR",pelvic_rotation)
                            print(pelvic_rotation)
#                             board.digital[TR0pin].write(1)
                        else:
#                             print("777")
                            print(pelvic_rotation)
#                             board.digital[TR0pin].write(0)
                    detect_state=1    #換腳偵測
                    block_count = 0
                
        else:
            block_count = block_count + 1
        
            
    while tt-ti-tl<samplingtime and getdata==1 and r!=apdm.APDM_NO_MORE_DATA:
        r= apdm.apdm_ctx_get_next_access_point_record_list(context)
        r= apdm.apdm_ctx_get_next_access_point_record_list(context)
        r= apdm.apdm_ctx_get_next_access_point_record_list(context)
        
        tt=time.clock()
    tl=tt-ti
    
# Disconnect
apdm.apdm_ctx_disconnect(context) 
# Free memory used by context
apdm.apdm_ctx_free_context(context)
context = None
print('Free Access Point Context Done.')
message = "END" + "\n"
print(message)
client_socket.sendall(message.encode())

In [ ]:
# Disconnect
apdm.apdm_ctx_disconnect(context) 
# Free memory used by context
apdm.apdm_ctx_free_context(context)
context = None
print('Free Access Point Context Done.')

In [ ]:
TRIAL_NAME = '\\20250306-1HS'
output_file = r"C:\Users\m2023\Desktop\experience_data"

In [ ]:
# foot gyro acc & orientation
# data_5610_csv=pd.DataFrame(data_5610)
# data_5611_csv=pd.DataFrame(data_5611)
# pelvic_angle_csv=pd.DataFrame(pelvic_angle)
index_min=min(len(data_5610),len(data_5611),len(pelvic_angle))
index_0=np.array([["","rightfoot","","","","",""]])
index=np.array([["time","ax","ay","az","wx","wy","wz"]])
aa=np.vstack([index_0,index,data_5610[1:index_min]])
index_0=np.array([["leftfoot","","","","",""]])
index=np.array([["ax","ay","az","wx","wy","wz"]])
bb=np.vstack([index_0,index,data_5611[1:index_min,1:7]])
index_0=np.array([["lumbar"]])
index=np.array([["pelvic_rotation"]])
cc=np.vstack([index_0,index,pelvic_angle[1:index_min]])
dd=np.hstack([aa,bb,cc])
dd_csv=pd.DataFrame(dd)
dd_csv.to_csv(output_file + TRIAL_NAME + 'Foot.csv',header=0,index=0)

# gyro, acc & orientation
index_min = len(data_5617_2)
index_0=np.array([["","lumbar","","","","","","","","",""]])
index=np.array([["time","ax","ay","az","wx","wy","wz","q0","q1","q2","q3"]])
aa=np.vstack([index_0,index,data_5617_2[1:index_min]])
aa_csv = pd.DataFrame(aa)
aa_csv.to_csv(output_file + TRIAL_NAME + 'Lumber.csv',header=0,index=0)

# online hS time
index_min=min(   len(R_HeelStrike_Detect_Time),  len(L_HeelStrike_Detect_Time)   )
index_0=np.array([["rightfoot"]])
ee = np.vstack([index_0,R_HeelStrike_Detect_Time[1:index_min]])
index_0=np.array([["leftfoot"]])
ff = np.vstack([index_0,L_HeelStrike_Detect_Time[1:index_min]])
gg = np.hstack([ee,ff])
gg_csv = pd.DataFrame(gg)
gg_csv.to_csv(output_file + TRIAL_NAME + 'hstime.csv',header=0,index=0)

# # 3 new OPALgyro, acc & orientation
# # index_min = min(len(data_7750),len(data_7770),len(data_11141))
# index_min=min(len(data_7750),len(data_5610_2))
# index_0=np.array([["","7750","","","","","","","","",""]])
# index=np.array([["time","ax","ay","az","wx","wy","wz","q0","q1","q2","q3"]])
# aa=np.vstack([index_0,index,data_7750[1:index_min]])
# index_0=np.array([["5610_2","","","","","","","","",""]])
# index=np.array([["ax","ay","az","wx","wy","wz","q0","q1","q2","q3"]])
# bb=np.vstack([index_0,index,data_5610_2[1:index_min,1:11]])
# # index_0=np.array([["7770","","","","","","","","",""]])
# # index=np.array([["ax","ay","az","wx","wy","wz","q0","q1","q2","q3"]])
# # bb=np.vstack([index_0,index,data_7770[1:index_min,1:11]])
# # index_0=np.array([["11141","","","","","","","","",""]])
# # index=np.array([["ax","ay","az","wx","wy","wz","q0","q1","q2","q3"]])
# # cc=np.vstack([index_0,index,data_11141[1:index_min,1:11]])
# dd=np.hstack([aa,bb])
# dd_csv=pd.DataFrame(dd)
# dd_csv.to_csv(r'C:\Users\LAB104_R229\Desktop\NDT NEW ERA' + TRIAL_NAME + 'NewOPAL.csv',header=0,index=0)

print('Save Data Done.')

In [ ]:
print(len(pelvic_angle))
print(np.mean(rotation_amplitude[2:len(rotation_amplitude)-1,0])) 
print(np.std(rotation_amplitude[2:len(rotation_amplitude)-1,0], ddof=1))   
pelvic_rotation_thres

In [ ]:
plt.plot(pelvic_angle[:])
data_now_orientation

In [ ]:
plt.plot(data_5610[:,5])
# plt.plot(data_5611[1000:1200,5])